-------------------------------------------- PARTE 1 -------------------------------------------

Modelo basico que responde a perguntas de python

Aqui podemos ver as dependencies que precisamos no nosso projeto.

In [ ]:
%pip install transformers torch torchvision huggingface_hub mediapipe

Depois vamos nos authneticar e carregar o modelo que queremos usar no nosso projeto, nesse caso aqui é o gemma 2B.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# Authentifiez-vous avec votre token Hugging Face
token = '' # por aqui o token do Hugging Face
login(token=token)

# Charger le modèle et le tokenizer
model_name = "google/gemma-2b"  # Nom du modèle GEMMA 2B
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Em seguida vamos declarar a nossa funçao de resposta que usa o que lhe mandamos em output para gerir uma resposta.

In [ ]:
def generate_response(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Aqui abaixo temos os nossos prompts/perguntas para fazer ao modelo.

In [ ]:
# Exemple de prompt pour analyse de code
prompt_code_analysis = "Analyze the following Python code:\n\n"
code_example = """
def add(a, b):
    return a + b

result = add(5, 3)
print(result)
"""
response = generate_response(prompt_code_analysis + code_example)
print(response)

# Exemple de prompt pour questions sur Python
prompt_question = "How to declare a variable in python language?\n\n"
response = generate_response(prompt_question)
print(response)






----------------------------- PARTE 2 ------------------------------







Aqui nesta segunda parte, nos vamos tentar fazer um segundo modelo onde treinamos esse mesmo para ter respostas mais concretas e detalhadas.

In [ ]:
# Instalação das bibliotecas necessárias
%pip install transformers datasets torch huggingface_hub pandas accelerate

Aqui vamos declarar todas as bibliotecas que necessitamos e entao declarar o nosso modelo.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset, Dataset
from huggingface_hub import login
import torch
import pandas as pd
import os
import zipfile

# Autenticação com o token Hugging Face
token = '' # por aqui o token do Hugging Face
login(token=token)

# Carregar o modelo e o tokenizer
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Converter os ficheiros recuperados do dataset em ficheiros json

O dataset que tentei usar é muito voluminozo entao esta aqui o link do dataset
https://www.kaggle.com/datasets/stackoverflow/pythonquestions

In [ ]:
# Définir les chemins de fichiers
zip_file_path = r'C:\Users\ricar\Downloads\archive.zip'  # Remplacez par le chemin de votre fichier ZIP
extract_dir = r'C:\Users\ricar\Documents\Aulas_Portugal\Inteligencia artificial\tema1'  # Remplacez par le chemin du répertoire d'extraction
json_dir = r'C:\Users\ricar\Documents\Aulas_Portugal\Inteligencia artificial\tema1\json'  # Remplacez par le chemin de sauvegarde du fichier JSON

# Créer le répertoire d'extraction et de sauvegarde JSON s'ils n'existent pas
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(json_dir, exist_ok=True)

# Extraire le fichier ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Lister les fichiers extraits
extracted_files = os.listdir(extract_dir)
print("Fichiers extraits:", extracted_files)

# Traiter chaque fichier CSV extrait
for file_name in extracted_files:
    if file_name.endswith('.csv'):
        csv_file_path = os.path.join(extract_dir, file_name)
        
        # Charger le fichier CSV
        df = pd.read_csv(csv_file_path, encoding='latin1')
        
        # Convertir le DataFrame en JSON
        json_file_path = os.path.join(json_dir, file_name.replace('.csv', '.json'))
        df.to_json(json_file_path, orient='records', lines=True)
        
        print(f'Dataset sauvegardé en JSON à {json_file_path}')

Carregar e preparar os fichieros para treino

In [ ]:
# Definir caminhos para os arquivos JSON
json_files = {
    "Answers": r"C:\Users\ricar\Documents\Aulas_Portugal\Inteligencia artificial\tema1\json\Answers.json",
    "Questions": r"C:\Users\ricar\Documents\Aulas_Portugal\Inteligencia artificial\tema1\json\Questions.json",
    "Tags": r"C:\Users\ricar\Documents\Aulas_Portugal\Inteligencia artificial\tema1\json\Tags.json"
}

# Carregar os arquivos JSON
answers_df = pd.read_json(json_files["Answers"], lines=True)
questions_df = pd.read_json(json_files["Questions"], lines=True)
tags_df = pd.read_json(json_files["Tags"], lines=True)

# Unir os dados conforme necessário (ajustar conforme sua necessidade)
merged_df = questions_df.merge(answers_df, on="Id")  # Exemplo de junção por "Id"

# Criar um Dataset do Hugging Face a partir do DataFrame
dataset = Dataset.from_pandas(merged_df)

# Dividir o dataset em treino e validação
dataset = dataset.train_test_split(test_size=0.1)

def tokenize_function(examples):
    return tokenizer(examples['Body'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
# trainer.save_model("caminho_para_salvar_seu_modelo")

# Avaliar o modelo
results = trainer.evaluate()
print(results)

Definiçao da funçao de geraçao e prompts de perguntas

In [ ]:
# Função para gerar resposta
def generate_response(prompt, max_length=150, temperature=0.7, top_p=0.9, repetition_penalty=2.0, no_repeat_ngram_size=2):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs, 
        max_length=max_length, 
        temperature=temperature, 
        top_p=top_p, 
        repetition_penalty=repetition_penalty, 
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Exemplo de prompt para análise de código
prompt_code_analysis = "Analyze the following Python code:\n\n"
code_example = """
def add(a, b):
    return a + b

result = add(5, 3)
print(result)
"""
response = generate_response(prompt_code_analysis + code_example)
print("Code Analysis Response:\n", response)

# Exemplo de prompt para perguntas sobre Python
prompt_question = "Explain how to declare a variable in Python.\n"
response = generate_response(prompt_question)
print("Question Response:\n", response)
print("End of the response\n")

# Teste com mais exemplos
prompt_question = "What is a function in Python?\n"
response = generate_response(prompt_question)
print("Question Response:\n", response)
print("End of the response\n")